<a href="https://colab.research.google.com/github/pavlyukm/ml-cp/blob/main/ML_Course_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, r2_score, explained_variance_score, mean_absolute_percentage_error
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_excel('/content/drive/MyDrive/ML Course Project/Цукровий Буряк.xlsx')

In [ ]:
weather_data = pd.read_excel('/content/drive/MyDrive/ML Course Project/Погода.xlsx')

In [ ]:
data = data.dropna(subset=['Врожай-\nність'])
data = data.rename(columns={'Врожай-\nність': 'Yield'})

In [ ]:
print(data.head())

  Зведений підрозділ  Підрозділ    Урочище   Рік Гібрид 1  \
0              Бишів      Бишів    Цап-лап  2016      NaN   
1              Бишів      Бишів  Сабанівка  2016      NaN   
2              Бишів      Бишів  Сабанівка  2016      NaN   
3             Жвирка      Гійче    Застава  2016      NaN   
4          Розворяни  Андріївка       Турє  2016      NaN   

  Технологія\n захисту Стійкість до нематоди Церкоспора  \
0             Класична                   NaN        NaN   
1             Класична                   NaN        NaN   
2             Класична                   NaN        NaN   
3             Класична                   NaN        NaN   
4             Класична                   NaN        NaN   

  Кореневі і/або нематода Сівозміна з буряком  ... Прогноз Цукру  \
0                     NaN                   4  ...           NaN   
1                     NaN                   4  ...           NaN   
2                     NaN                   4  ...           NaN   
3     

# Data Description

## Dataset Overview
- **Description**: This dataset contains information about sugar beets farming, including sugar yields and associated weather conditions. The goal is to analyze the impact of various factors on sugar yields.

## Dataset Structure
- **Number of Records**: 962
- **Number of Features**: 62
- **File Format**: XLSX

## Feature Description

| Feature Name                | Description                                                                 |
|------------------------------|-----------------------------------------------------------------------------|
| Зведений підрозділ           | Потребує уточнення, швидш за все не важлива для нас змінна                                                     |
| Підрозділ                    | Потребує уточнення, швидш за все не важлива для нас змінна                                                           |
| Урочище                      | Основна ділянка ферми, по якій будується звіт: всі наступні дані стсосуються саме урочища                                           |
| Рік                          | рік збору врожаю                                                    |
| Гібрид 1                     | тип буряка: змінна для word embedding?                                                     |
| Технологія захисту           | Класична або СМАРТ. Слід заенкодити в 1 \ 0                                                |
| Стійкість до нематоди        | пусто = ні = 0, так = 1                                                     |
| Церкоспора                   | -, --, +, ++, або цифри -- потребує уточнення, невідомо як енкодити                                                  |
| Кореневі і/або нематода      | присутній, -, +, ++, --, +н, ++/-н, ++н, -/-, -н, або цифри, потребує уточнення, невідомо як енкодити                                                     |
| Сівозміна з буряком          | 3, 4, більше 4, Більше 4 (Нові площі/облоги), потребує уточнення                                               |
| Примітки                     | швидш за все неважлива змінна, досить спарс, але можна уточнити                                                    |
| Попередник                   | яка культура росла перед буряком. Також можна зробити ебмеддінг?                                                   |
| Згрупований попередник       | потрібно уточнити, швидш за все неважливо                                                  |
| Площа                        | float, гектари                                                  |
| Осн. Культура                | Цукровий буряк                                                    |
| Період грунтообробітку       | зима або осінь, заенкодити 1-0                                                     |
| К-ть основних обробітків      | 1 слід або 2 сліди: 1 \ 2                                                   |
| Тип грунту                   | тут не робоча VLOOKUP функція, потрібно попросити ці дані додатково                                                 |
| N                            | int: припускаю це мг \ га                                                     |
| P2O5                         | int: припускаю це мг \ га                                                     |
| K2O                          | int: припускаю це мг \ га                                                  |
| Na2O                         | int: припускаю це мг \ га                                                    |
| S                            | int: припускаю це мг \ га                                                    |
| Безводний Аміак              | порожньо = 0 або безводний = 1                                                  |
| Сидерати                     | порожньо = 0 так = 1                                                   |
| Органіка                     | ні \ так = 0 1                                                  |
| Вид органіки                 | ВРХ, СВИ, КУР, КОМП, багато порожнього = енкодити                                                  |
| Норма                        | int: невідомо, чого це стосується                                                |
| Негативні чинники (Враховувати)| Гнилі, не рахуємо, слабкі сходи або порожньо = енкодити                                                |
| Дата посіву                  | дата посіву у різному форматі                                                 |
| Середня дата посіву          | 1-12 місяць посіву                                                  |
| Діапазон посіву              | string: крайня дата посіву                                                   |
| Дата появи сходів            | date                                               |
| Дата змикання міжряддя       | дата зикання в різному форматі, швидш за все важлива змінна                                              |
| Дата збирання                | date: дата збирання врожаю                                                  |
| Густота на 30 день, тис.     | float: потрібно уточнити, що це означає                                              |
| Норма висіву                 | 120 або порожньо: потрібно уточнити                                                   |
| Густота збирання             | float: густота на момент збору? Потрібно уточнити                                                   |
| Польова схожість             | 0-100%, густота збирання \ норма висіву                                      |
| Тривалість сходів            | int: дата появи сходів - дата посіву                                                   |
| Днів до змикання міжряддя    | int: дата змикання міжряддя - дата появи сходів                                                  |
| Тривалість активної вегетації| float: дата збирання - дата змикання                                     |
| Група активної вегет         |string: часові періоди в днях, заенкодити                                                 |
| Період сходів                | string: часові періоди в днях, заенкодити                                                |
| Період вегетації             | date: дата збирання - дата появи сходів                                                 |
| Група періоду вегетації      | string: часові періоди в днях, заенкодити                                                       |
| Врожай-                 | float                                                    |
| Базова врожайність           | float                                                     |
| Відносна врожайність         | float: врожай- \ базова врожайність                                                  |
| Цукристість                  | float                                               |
| Вал Цукру                    | float: Вихід Цукру з га  \ Площа                                               |
| Вихід Цукру з га             | float: цукристіть \ Врожай-                                                 |
| Прогноз Цукру                | float: =0.0427*тривалість активної вегетації + 6.9375                                                     |
| Відхилення по цукру          | float, % : Вихід Цукру з га \ Прогноз Цукру - 1                                                     |
| Вал                          | float: Врожай- * Площа                                                 |
| Вал Відносної врожайності    | float: Відносна врожайність * Площа                                             |
| NUE                          | float: Врожай- \  N *1000                                                    |
| Вивезення днів               | int                                                   |
| Меляса                       | int                                                   |
| рН                           | неробочий VLOOKUP, треба уточнити дані                                                   |
| Дефекат                      | 0 = порожньо = ні, 1 = так                                                    |
| Періоди вивезення            | string: часові періоди в днях, заенкодити                                                     |


## Target Variable
- **Name**: Врожай-
- **Type**: Float
- **Description**: Врожай цукру з 1 гектару


# Data Processing Pipeline

In [ ]:
print(data.columns)

Index(['Зведений підрозділ', 'Підрозділ', 'Урочище', 'Рік', 'Гібрид 1',
       'Технологія\n захисту', 'Стійкість до нематоди', 'Церкоспора',
       'Кореневі і/або нематода', 'Сівозміна з буряком', 'Примітки',
       'Попередник', 'Згрупований попередник', 'Площа', 'Осн. Культура',
       'Період грунтообробітку', 'К-ть основних обробітків', 'Тип грунту', 'N',
       'P2O5', 'K2O', 'Na2O', 'S', 'Безводний Аміак', 'Сидерати', 'Органіка',
       'Вид органіки', 'Норма', 'Негативні чинники (Враховувати)',
       'Дата посіву', 'Середня дата посіву', 'Діапазон посіву',
       'Дата появи сходів', 'Дата змикання міжряддя', 'Дата збирання',
       'Густота на 30 день, тис.', 'Норма висіву', 'Густота збирання',
       'Польова схожість', 'Тривалість сходів', 'Днів до змикання міжряддя',
       'Тривалість активної вегетації', 'Група активної вегет',
       'Період сходів', 'Період вегетації', 'Група періоду вегетації', 'Yield',
       'Базова врожайність', 'Відносна врожайність', 'Цукристіст

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd


label_encoder = LabelEncoder()
data_processed = data.copy()

columns_to_drop = ['Згрупований попередник', 'Тип грунту', 'Осн. Культура', 'рН', 'Примітки', 'Зведений підрозділ', 'Підрозділ', 'К-ть основних обробітків', 'Норма']
data_processed = data_processed.drop(columns=columns_to_drop, errors='ignore')

numeric_features = ['Yield', 'Відносна врожайність', 'Вал Цукру',
                    'Вихід Цукру з га', 'Прогноз Цукру', 'Відхилення по цукру',
                    'Вал', 'Вал Відносної врожайності', 'NUE']

for feature in numeric_features:
    if feature in data_processed.columns:
        data_processed[feature] = pd.to_numeric(data_processed[feature], errors='coerce')

categorical_features = ['Урочище', 'Гібрид 1',
                        'Технологія\n захисту', 'Стійкість до нематоди', 'Церкоспора',
                        'Кореневі і/або нематода', 'Сівозміна з буряком',
                        'Попередник', 'Період грунтообробітку',
                        'Безводний Аміак', 'Сидерати', 'Органіка', 'Вид органіки',
                        'Негативні чинники (Враховувати)', 'Середня дата посіву',
                        'Діапазон посіву', 'Днів до змикання міжряддя', 'Тривалість активної вегетації',
                        'Група активної вегет', 'Період сходів', 'Група періоду вегетації',
                        'Дефекат', 'Періоди вивезення']

for feature in categorical_features:
    if feature in data_processed.columns:
        data_processed[feature] = data_processed[feature].astype(str).fillna('missing')
        data_processed[feature] = label_encoder.fit_transform(data_processed[feature])

date_columns = ['Дата посіву', 'Дата появи сходів', 'Дата змикання міжряддя', 'Дата збирання', 'Період вегетації']

for feature in date_columns:
    if feature in data_processed.columns:
        data_processed[feature] = pd.to_datetime(data_processed[feature], errors='coerce')
        data_processed[feature + '_year'] = data_processed[feature].dt.year
        data_processed[feature + '_month'] = data_processed[feature].dt.month
        data_processed[feature + '_day'] = data_processed[feature].dt.day
        data_processed[feature + '_dayofweek'] = data_processed[feature].dt.dayofweek
        data_processed = data_processed.drop(feature, axis=1)

print(data_processed.head())
print(data_processed.dtypes)

   Урочище   Рік  Гібрид 1  Технологія\n захисту  Стійкість до нематоди  \
0      161  2016         1                     0                      0   
1      132  2016         1                     0                      0   
2      132  2016         1                     0                      0   
3       60  2016         1                     0                      0   
4      152  2016         1                     0                      0   

   Церкоспора  Кореневі і/або нематода  Сівозміна з буряком  Попередник  \
0          15                       23                    2           6   
1          15                       23                    2           2   
2          15                       23                    2           2   
3          15                       23                    2          10   
4          15                       23                    2           4   

   Площа  ...  Дата змикання міжряддя_day  Дата змикання міжряддя_dayofweek  \
0  181.0  ...      

In [ ]:
data_processed = data_processed.dropna(subset=['Yield'])
X = data_processed.drop('Yield', axis=1)
y = data_processed['Yield']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# random forest to get started
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Feature importance
importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print(feature_importance_df)

Mean Squared Error: 1.0540474925480672
                 Feature  Importance
37      Вихід Цукру з га    0.849311
34  Відносна врожайність    0.058969
35           Цукристість    0.030174
42                   NUE    0.023004
39   Відхилення по цукру    0.011951
..                   ...         ...
3   Технологія\n захисту    0.000024
45               Дефекат    0.000015
15                     S    0.000012
17              Сидерати    0.000006
24          Норма висіву    0.000000

[67 rows x 2 columns]


In [ ]:
threshold = 0.01
important_features = feature_importance_df[feature_importance_df['Importance'] > threshold]['Feature']

X_important = X[important_features]

X_train_important, X_test_important, y_train, y_test = train_test_split(X_important, y, test_size=0.25, random_state=42)

rf_model_important = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model_important.fit(X_train_important, y_train)

y_pred_important = rf_model_important.predict(X_test_important)

mse_important = mean_squared_error(y_test, y_pred_important)
print(f'Mean Squared Error with important features: {mse_important}')

Mean Squared Error with important features: 0.5649684492788387


In [ ]:
xgboost_model = xgb.XGBRegressor(eval_metric='rmse', random_state = 42)

# grid for hyperparameter tuning
param_grid = {"max_depth":    [5],
              "n_estimators": [1200, 2500, 5000],
              "learning_rate": [0.02, 0.03],
              "subsample" : [0.5]
              }

grid_search = GridSearchCV(estimator=xgboost_model, param_grid=param_grid, cv=5)

grid_search.fit(X_train_important, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

y_pred_xgb = best_model.predict(X_test_important)

mse_xgb = mean_squared_error(y_test, y_pred_xgb)
mae = mean_absolute_error(y_test, y_pred_xgb)
rmse = np.sqrt(mse_xgb)
r2 = r2_score(y_test, y_pred_xgb)
explained_variance = explained_variance_score(y_test, y_pred_xgb)
mape = mean_absolute_percentage_error(y_test, y_pred_xgb)
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

# Ensure that the values are positive to avoid log errors
y_test_pos = np.where(y_test <= 0, 1e-10, y_test)
y_pred_xgb_pos = np.where(y_pred_xgb <= 0, 1e-10, y_pred_xgb)
rmsle_value = rmsle(y_test_pos, y_pred_xgb_pos)
print("The best hyperparameters are ",grid_search.best_params_)
print(f'Mean Squared Error with XGBoost: {mse_xgb}')
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'RMSLE: {rmsle_value}')
print(f'R-squared: {r2}')
print(f'Explained Variance Score: {explained_variance}')
print(f'MAPE: {mape}')

The best hyperparameters are  {'learning_rate': 0.02, 'max_depth': 5, 'n_estimators': 5000, 'subsample': 0.5}
Mean Squared Error with XGBoost: 0.4395826177030723
MAE: 0.2976454965884867
RMSE: 0.6630102696814524
RMSLE: 0.011097636237821172
R-squared: 0.9967989485119624
Explained Variance Score: 0.9968059588550209
MAPE: 0.0047103340935051976
